# Langgraph Prebuilt

This noteboook contains the code to run langgraph agent with prebuilt architectures, we look into the React Agents


In [19]:
# Load the environment variables
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [20]:
# Tools used by the agent
from langchain_core.tools import tool


@tool
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

In [21]:
from langgraph.errors import GraphRecursionError
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel


class WeatherResponse(BaseModel):
    """The response from the weather agent."""

    conditions: str


# Create the agent using the reasoning action agent design
agent = create_react_agent(
    model="openai:gpt-4o",
    tools=[get_weather],
    response_format=WeatherResponse,
    prompt="You are a helpful assistant",
)

# Set recursion limits to avoid infinite loop
max_iterations = 3
recursion_limit = 2 * max_iterations + 1
agent_with_recursion_limit = agent.with_config(recursion_limit=recursion_limit)

In [22]:
import logging

logger = logging.getLogger("langgraph")
logger.setLevel(logging.DEBUG)

# Run the agent
try:
    # The response returned will be the final output of the agent
    response = agent_with_recursion_limit.invoke(
        {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
    )
except GraphRecursionError:
    logger.error("Agent stopped due to max iterations.")

# This is how langgraph returns the structured output
response["structured_response"]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


WeatherResponse(conditions="As of the latest update, San Francisco is experiencing mild and pleasant weather with clear skies and an abundance of sunshine. Here's a detailed overview:\n\n- **Temperature:** The day temperature hovers around a comfortable range, making it ideal for outdoor activities. Expect daytime highs around 68-72°F (20-22°C) and cooler evenings around 55-59°F (13-15°C).\n  \n- **Sky Conditions:** The sky is primarily clear with minimal cloud cover, allowing for bright and sunny conditions throughout the day.\n  \n- **Wind:** A gentle breeze coming from the west at around 5-10 mph, providing a refreshing sensation without being too chilly.\n\n- **Humidity:** Mildly humid, maintaining a comfortable level for most people, given the wind and temperature balance.\n\nWhether you're planning to explore the city's iconic landmarks, stroll along the beach, or visit the vibrant neighborhoods, it's the perfect day to be out and about in San Francisco!\n\n**Note**: Always check

In [24]:
# If you want to see the updates of the Agent at every stage we need to
# stream the updates between the LLM's and tools as done below
for chunk in agent_with_recursion_limit.stream(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    stream_mode="updates",
):
    logger.info(chunk)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:langgraph:{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_iOjakeKLgi7oaCwgNJLowu3y', 'function': {'arguments': '{"city":"San Francisco"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 56, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BWChGxu4mP3P20iFhWLCRFDG88nNw', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d42b6ee0-1059-4f55-ba26-367e047afd72-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_iOjakeKLgi7oaCwgN